In [17]:
import sympy as sym
import math as m
import numpy as np
import pandas as pd

import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import SIC_POVM_functions as sic

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

np.set_printoptions(linewidth=150)
def pp(M, rounding=3):
    if type(M) is not np.ndarray:
        print(M)
    else:
        M = M.real.round(rounding) + 1j*M.imag.round(rounding)
        print(np.round(M, rounding))


In [7]:
w  = np.exp(2*np.pi*1j/3)

A_T = np.array([[0, 1, -1], [-1, 0, 1], [1, -1, 0]])
A = np.transpose(A_T)

pp(A)


[[ 0.+0.j -1.+0.j  1.+0.j]
 [ 1.+0.j  0.+0.j -1.+0.j]
 [-1.+0.j  1.+0.j  0.+0.j]]


In [10]:
N =3
# Basis states |x> are the columns of A
basis_states = np.array([[1,0,0], [0,1,0], [0,0,1]])

# Construct xx matrix
xx = np.array([x.reshape(1,3).T @ x.reshape(1,3) for x in basis_states])
x1_x = np.array([basis_states[(i+1)%N].reshape(1,3).T @ basis_states[i].reshape(1,3) for i in range(N)])

# Construct x_1,x matrix


# verified both xx and x1_x 
w_matrix = np.array([w**i for i in range(N)])

W_array = np.array([ w_matrix[i]*xx[i] for i in range(N)])

V, W = np.sum(x1_x, axis=0, keepdims=True), np.sum(W_array, axis=0, keepdims=True)



In [11]:
# checking it they recreate cA


a1 = V - V@V
a2 = w*V@W - w**2*V@V@W
a3 = w**2 * V @ W @ W - w*V@V@W@W

a4 = (V+V@V)
a5 = w*V@W + w**2*V@V@W
a6 = w**2 * V @ W @ W + w*V@V@W@W

a7 = W
a8 = W@W
a9 = np.eye(3)

a1, a2, a3, a4, a5, a6, a7, a8, a9 = a1.reshape(3,3), a2.reshape(3,3), a3.reshape(3,3), a4.reshape(3,3), a5.reshape(3,3), a6.reshape(3,3), a7.reshape(3,3), a8.reshape(3,3), a9.reshape(3,3)


In [12]:
A1, A2, A3 = (1/np.sqrt(6))*a1, (1/np.sqrt(6))*a2, (1/np.sqrt(6))*a3
A4, A5, A6 = (1/np.sqrt(6))*a4, (1/np.sqrt(6))*a5, (1/np.sqrt(6))*a6
A7, A8, A9 = (1/np.sqrt(3))*a7, (1/np.sqrt(3))*a8, (1/np.sqrt(3))*a9

In [20]:
# horizontal stack three matrices
h1A = np.hstack((A1, A2, A3))
h2A = np.hstack((A4, A5, A6))
h3A = np.hstack((A7, A8, A9))

# vertical stack the three matrices

M = np.vstack((h1A, h2A, h3A))

pp(M, 4)

[[ 0.    +0.j     -0.4082+0.j      0.4082+0.j      0.    +0.j     -0.4082+0.j      0.4082+0.j      0.    +0.j     -0.4082+0.j      0.4082+0.j    ]
 [ 0.4082+0.j      0.    +0.j     -0.4082+0.j     -0.2041+0.3536j  0.    +0.j      0.2041-0.3536j -0.2041-0.3536j  0.    +0.j      0.2041+0.3536j]
 [-0.4082+0.j      0.4082+0.j      0.    +0.j      0.2041+0.3536j -0.2041-0.3536j  0.    +0.j      0.2041-0.3536j -0.2041+0.3536j  0.    +0.j    ]
 [ 0.    +0.j      0.4082+0.j      0.4082+0.j      0.    +0.j      0.4082+0.j      0.4082+0.j      0.    +0.j      0.4082+0.j      0.4082+0.j    ]
 [ 0.4082+0.j      0.    +0.j      0.4082+0.j     -0.2041+0.3536j  0.    +0.j     -0.2041+0.3536j -0.2041-0.3536j  0.    +0.j     -0.2041-0.3536j]
 [ 0.4082+0.j      0.4082+0.j      0.    +0.j     -0.2041-0.3536j -0.2041-0.3536j  0.    +0.j     -0.2041+0.3536j -0.2041+0.3536j  0.    +0.j    ]
 [ 0.5774+0.j      0.    +0.j      0.    +0.j      0.5774+0.j      0.    +0.j      0.    +0.j      0.5774+0.j      0. 

In [23]:
pp(np.conjugate(M.T) @ M )       # multiplying with its conjugate transpose. Gives identity matrix.

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j  1.+0.j -0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [-0.+0.j  0.+0.j  0.+0.j -0.+0.j  1.+0.j -0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j  1.+0.j  0.+0.j -0.+0.j  0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j -0.+0.j]
 [-0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j -0.+0.j -0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j -0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j -0.+0.j -0.+0.j  1.+0.j]]


In [26]:
# check if a matrix is unitary using sympy
M_sym = sym.Matrix(M)
M_sym.is_unitary


AttributeError: 'MutableDenseMatrix' object has no attribute 'is_unitary'

#### Trying the Zeilinger decomposition

In [49]:
import numpy as np
import strawberryfields
from strawberryfields.decompositions import rectangular

# Define a 2x2 identity matrix
# identity_matrix = np.eye(2)
# hadamard  = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
SIC_POVM_extension_dov  = M

# Decompose the identity matrix into beam splitters and phase shifters
tlist, localV, _ = rectangular(SIC_POVM_extension_dov)

# Print the results
print("T Matrices List (Beam Splitters):")
print(tlist)
print("Local V (Phase Shifters):")
print(localV)

T Matrices List (Beam Splitters):
[[0, 1, 0, 0, 9], [2, 3, 1.5707963267948966, 0, 9], [1, 2, 1.5707963267948966, 0, 9], [0, 1, 0.7853981633974483, 0.0, 9], [4, 5, 0.7853981633974482, -1.0471975511965972, 9], [3, 4, 0.9553166181245093, -1.0471975511965974, 9], [2, 3, 1.0471975511965979, 6.841059687549134e-16, 9], [1, 2, 1.5707963267948966, 2.101228658090909, 9], [0, 1, 0.9553166181245093, -2.094395102393195, 9], [6, 7, 0.7853981633974483, 5.828670879282075e-16, 9], [5, 6, 1.1071487177940906, -3.141592653589793, 9], [4, 5, 1.318116071652818, -0.0, 9], [3, 4, 1.1333150770761202, -1.7609219301413637, 9], [2, 3, 1.1970041519603862, 1.0403639954988846, 9], [1, 2, 1.318116071652818, -2.6179938779914944, 9], [0, 1, 0.6154797086703875, -2.6179938779914944, 9]]
Local V (Phase Shifters):
[ 0.5       +8.66025404e-01j -0.38867167+9.21376326e-01j
 -0.5       +8.66025404e-01j  1.        -4.81906560e-16j
  0.75592895+6.54653671e-01j -0.5       +8.66025404e-01j
  0.8660254 -5.00000000e-01j -0.5       +